In [1]:
import numpy as np
from numpy import nan
import matplotlib.pyplot as plt
import pandas as pd
pd.options.mode.chained_assignment = None 
from sklearn.preprocessing import OneHotEncoder

import nltk
import re
import string
from nltk.corpus import stopwords
from nltk import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial import distance

In [2]:
rel_que = pd.read_csv('data/relevant_questions.csv')
ans = pd.read_csv('data/keyword_answer.csv')
kdf = pd.read_csv('data/keywords.csv')

In [3]:
df1 = pd.read_csv('data/original_data1.csv')
df2 = pd.read_csv('data/original_data2.csv')
df3 = pd.read_csv('data/original_data3.csv')
df4 = pd.read_csv('data/original_data4.csv')
df5 = pd.read_csv('data/original_data5.csv')
df = pd.concat([df1,df2,df3,df4,df5], axis=0)
df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.reset_index(drop=True, inplace=True)
from bs4 import BeautifulSoup
df.drop(columns=['answers', 'tags'], axis=1, inplace = True)

In [4]:
def length_text(text):
    if(type(text) != type(0.0)):
        text = text.split(' ')
        return len(text)
    else:
        return 0
ans['ans length'] = ans['answer'].apply(length_text)

In [5]:
rel_ans = pd.DataFrame()
for i in enumerate(rel_que['id']):
    rel_ans = pd.concat([ans[ans['id'] == i[1]], rel_ans])

In [6]:
topics = ['Topic 0', 'Topic 1', 'Topic 2', 'Topic 3', 'Topic 4', 'Topic 5', 'Topic 6', 'Topic 7']

In [7]:
def get_dataframe(topic_list):
    if(type(topic_list) == type(0.0)):
        return pd.Series([0,0,0,0,0,0,0,0])
    else:
        topic_list = topic_list.split(', ')
        tl = []
        for topic in topics:
            if(topic in topic_list):
                val = 1
            else: 
                val = 0
            tl.append(val)
        return pd.Series(tl)

In [8]:
rel_ans[topics] = rel_ans['topic list'].apply(get_dataframe)
rel_ans.drop(columns=['topic list', 'id'], inplace = True)
rel_ans['answer list'] = rel_ans['answer'] + rel_ans['code']

In [9]:
que_ans = pd.DataFrame()
for i in enumerate(rel_que['id']):
    que_ans = pd.concat([df[df['id'] == i[1]], que_ans])

In [10]:
def extract_text(text):
    soup = BeautifulSoup(text, 'lxml')
    txt = "".join([txt.text for txt in soup.find_all("p")])
    return txt

que_ans['body'] = que_ans['body'].apply(extract_text)
que_ans['question list'] = que_ans['title'] + que_ans['body']
que_ans.drop(columns=['title', 'body'], axis=1, inplace = True)

In [11]:
rel_que

,id,questions,score
0,48641350,How to merge two lists into a list of multiple...,0.799961
1,46362972,Merge list of lists in python 3,0.785947
2,1158128,Merge sorted lists in python,0.750189
3,44476206,how to merge two list having dict,0.749920
4,58971955,How to merge two dataframe?,0.709934
...,...,...,...
95,14008075,How to merge many to many relations from one d...,0.372127
96,55206146,Merge Pandas Dataframe with non-unique index w...,0.366641
97,58678560,How to merge similar data into a custom field ...,0.365843
98,50727548,How to merge multiple dataframes with the same...,0.354777


In [12]:
rel_ans

,answer,link,code,ans length,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,answer list
4728,You can either use update:output:Or loc:output:,NaN,update\r\nEmpty_DF.update(ROI_DF)\r\n\r\n ...,6,0,0,0,0,0,0,0,1,You can either use update:output:Or loc:output...
4729,"This is perfect case for DataFrame.update, whi...",https://pandas.pydata.org/pandas-docs/stable/r...,DataFrame.update\r\nEmpty_DF.update(ROI_DF)\r\...,44,1,0,0,1,0,0,0,0,"This is perfect case for DataFrame.update, whi..."
4730,In your case reindex_like,NaN,reindex_like\r\nyourdf=ROI_DF.reindex_like(Emp...,4,0,0,0,0,0,0,0,0,In your case reindex_likereindex_like\r\nyourd...
60589,You can put the different column into index an...,NaN,"pd.concat([plantsFrame.set_index(['plants']), ...",12,0,0,0,0,0,0,0,1,You can put the different column into index an...
27294,Your desired outputs are no longer really mode...,https://stackoverflow.com/a/35019122/12197595,NaN,46,0,1,0,0,1,0,0,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39444,Use this code :Ouptut:,NaN,"list1= [['user1', 186, 'Feb 2017, Apr 2017', 5...",4,0,0,0,1,0,0,1,0,"Use this code :Ouptut:list1= [['user1', 186, '..."
18975,This might help,NaN,"desiredlist = list(map(lambda y:[lst1,y],lst2)...",3,0,0,0,0,0,0,0,0,This might helpdesiredlist = list(map(lambda y...
18976,You could achieve your desired output with ite...,NaN,itertools.zip_longest\r\nfillvalue\r\n>>> from...,31,0,0,0,0,0,0,0,1,You could achieve your desired output with ite...
18977,"Or you can use use append, but you need to cre...",NaN,lst3 = []\r\nfor elem in lst2:\r\n theNew =...,16,1,0,0,0,0,0,0,1,"Or you can use use append, but you need to cre..."


In [13]:
que_ans

,id,score,question list
4728,58276538,2,How to merge/join empty dataframe with another...
4729,58276538,3,How to merge/join empty dataframe with another...
4730,58276538,1,How to merge/join empty dataframe with another...
60589,50727548,0,How to merge multiple dataframes with the same...
27294,58678560,0,How to merge similar data into a custom field ...
...,...,...,...
39444,46362972,1,Merge list of lists in python 3I have this 2 l...
18975,48641350,0,How to merge two lists into a list of multiple...
18976,48641350,1,How to merge two lists into a list of multiple...
18977,48641350,1,How to merge two lists into a list of multiple...


In [14]:
stop_words = stopwords.words('english')
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()

In [15]:
def text_process(text):
    if(type(text) != type(0.0)):
        text = text.lower()
        text = re.sub("[^a-z]", " ", text)
        text = text.strip()
        token = word_tokenize(text)
        text = [i for i in token if not i in stop_words]
        output = []
        for word in text:
            output.append(stemmer.stem(word))
        text = []
        for word in output:
            text.append(lemmatizer.lemmatize(word))
        text = " ".join(text)
    return text

In [16]:
que_ans['question list'] = que_ans['question list'].apply(text_process)
rel_ans['answer list'] = rel_ans['answer list'].apply(text_process)

In [35]:
data = rel_ans.join(que_ans)

In [18]:
def cosine_score(question, answer):
    if(type(answer) == type(0.0)):
        return 0
    else:
        sentences = [question, answer]
        vectorizer = CountVectorizer()
        vector = vectorizer.fit_transform(sentences)
        text1 = vector.toarray()[0].tolist()
        text2 = vector.toarray()[1].tolist()
        cosc = 1-distance.cosine(text1, text2)
        return cosc
    
def entropy(text, tfidf_dict):
    if(type(text) == type(0.0)):
        return 0
    else:
        token = word_tokenize(text)
        entropy = 0.0
        for word in token:
            try:
                entropy = entropy + tfidf_dict[word]
            except:
                entropy = entropy + 0.0
        return entropy
    
def query_sim_score(query, answer):
    score = 0
    if(type(answer) == type(0.0)):
        return 0
    else:
        query = query
        for q in query:
            if(q in answer):
                score =  score + 1
        score = score/len(query)
        return score

In [19]:
query = 'merge two lists in python'
query = text_process(query)

In [20]:
docs = data['answer list'].dropna().tolist()
cv = CountVectorizer(stop_words = stop_words)
word_count_vector = cv.fit_transform(docs)

In [21]:
tfidf_transformer = TfidfTransformer(smooth_idf=True,use_idf=True)
tfidf_transformer.fit_transform(word_count_vector)

<157x1345 sparse matrix of type '<class 'numpy.float64'>'
	with 4237 stored elements in Compressed Sparse Row format>

In [22]:
tfidf_array = tfidf_transformer.idf_
words = cv.get_feature_names()
tfidf_dict = {}
for i in range(0, len(tfidf_transformer.idf_)):
    tfidf_dict[words[i]] = tfidf_array[i]

In [23]:
def normalised_score():
    sc = ((score - min_score)/(max_score - min_score))
    return sc

In [24]:
def score(user_score, cosine_score, entropy_score, sim_score, min_user_score, max_user_score, min_cosine_score, max_cosine_score, min_entropy_score, max_entropy_score):
    if(min_user_score == max_user_score):
        user_score = user_score - min_user_score
    else:
        user_score = ((user_score - min_user_score)/(max_user_score - min_user_score))
        
    if(min_cosine_score == max_cosine_score):
        cosine_score = cosine_score - min_cosine_score
    else:
        cosine_score = ((cosine_score - min_cosine_score)/(max_cosine_score - min_cosine_score))
        
    if(min_entropy_score == max_entropy_score):
        entropy_score = entropy_score - min_entropy_score
    else:
        entropy_score = ((entropy_score - min_entropy_score)/(max_entropy_score - min_entropy_score))
    return (user_score + cosine_score + entropy_score + 2 * sim_score)

In [25]:
data

,answer,link,code,ans length,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,answer list,id,score,question list
4728,You can either use update:output:Or loc:output:,NaN,update\r\nEmpty_DF.update(ROI_DF)\r\n\r\n ...,6,0,0,0,0,0,0,0,1,either use updat output loc output updat empti...,58276538,2,merg join empti datafram anoth fill datafram e...
4729,"This is perfect case for DataFrame.update, whi...",https://pandas.pydata.org/pandas-docs/stable/r...,DataFrame.update\r\nEmpty_DF.update(ROI_DF)\r\...,44,1,0,0,1,0,0,0,0,perfect case datafram updat align indicesoutpu...,58276538,3,merg join empti datafram anoth fill datafram e...
4730,In your case reindex_like,NaN,reindex_like\r\nyourdf=ROI_DF.reindex_like(Emp...,4,0,0,0,0,0,0,0,0,case reindex likereindex like yourdf roi df re...,58276538,1,merg join empti datafram anoth fill datafram e...
60589,You can put the different column into index an...,NaN,"pd.concat([plantsFrame.set_index(['plants']), ...",12,0,0,0,0,0,0,0,1,put differ column index concat output pd conca...,50727548,0,merg multipl datafram number column one datafr...
27294,Your desired outputs are no longer really mode...,https://stackoverflow.com/a/35019122/12197595,NaN,46,0,1,0,0,1,0,0,1,NaN,58678560,0,merg similar data custom field use serial djan...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39444,Use this code :Ouptut:,NaN,"list1= [['user1', 186, 'Feb 2017, Apr 2017', 5...",4,0,0,0,1,0,0,1,0,use code ouptut list user feb apr user mai use...,46362972,1,merg list list python list list desir output f...
18975,This might help,NaN,"desiredlist = list(map(lambda y:[lst1,y],lst2)...",3,0,0,0,0,0,0,0,0,might helpdesiredlist list map lambda lst lst,48641350,0,merg two list list multipl list given two list...
18976,You could achieve your desired output with ite...,NaN,itertools.zip_longest\r\nfillvalue\r\n>>> from...,31,0,0,0,0,0,0,0,1,could achiev desir output itertool zip longest...,48641350,1,merg two list list multipl list given two list...
18977,"Or you can use use append, but you need to cre...",NaN,lst3 = []\r\nfor elem in lst2:\r\n theNew =...,16,1,0,0,0,0,0,0,1,use use append need creat new copi lst lst ele...,48641350,1,merg two list list multipl list given two list...


In [180]:
for topic in topics:
    topic_data = data[data[topic]==1][['id', 'answer', 'link', 'code', 'score', 'answer list', 'question list']]
    topic_data.dropna()
    entropy_score = []
    topic_df = pd.DataFrame()
    for i in range(0, topic_data.shape[0]):
        cosc = cosine_score(topic_data.iloc[i]['question list'], topic_data.iloc[i]['answer list'])
        entro = entropy(topic_data.iloc[i]['answer list'], tfidf_dict)
        sim_score = query_sim_score(query, topic_data.iloc[i]['answer list'])
        topic_df = topic_df.append(pd.Series([topic_data.iloc[i]['id'], topic_data.iloc[i]['answer'], topic_data.iloc[i]['link'], topic_data.iloc[i]['code'], topic_data.iloc[i]['score'], cosc, entro, sim_score]), ignore_index=True)
    if topic_df.shape[0]==0:
            continue
    else:
        topic_df.columns = ['id', 'answer', 'link', 'code', 'score', 'cosine score', 'entropy','sim score']
        min_user_score = topic_df['score'].min()
        max_user_score = topic_df['score'].max()
        min_cosine_score = topic_df['cosine score'].min()
        max_cosine_score = topic_df['cosine score'].max()
        min_entropy_score = topic_df['entropy'].min()
        max_entropy_score = topic_df['entropy'].max()
        scores = []
        for i in range(0, topic_df.shape[0]):
            scores.append(score(topic_df.iloc[i]['score'], topic_df.iloc[i]['cosine score'], topic_df.iloc[i]['entropy'], 
                                topic_df.iloc[i]['sim score'], min_user_score, max_user_score, min_cosine_score, max_cosine_score,
                                min_entropy_score, max_entropy_score))
        topic_df.drop(columns = ['score', 'cosine score', 'entropy'], inplace=True)
        topic_df['sc'] = scores
        topic_df.sort_values(by = 'sc', ascending=False, inplace=True, kind='quicksort')
        path = 'data/'+ topic +'.csv'
        topic_df = topic_df.head(15)
        topic_df.to_csv(path, index=False)

In [196]:
df = pd.read_csv("data/Topic 0.csv")['answer']

In [197]:
for ans in df:
    print("Hey ", ans)

Hey  There are many ways to do this, staying in Pandas I did the following.With the file structureThis code will work, it's a little verbose for explanation but you can shorten with implementation.
Hey  Ok, here's some working code. Is this roughly what you need?
I'm not too happy with it yet, it looks a bit ugly imho but I want to know if it's the right direction.Output:
Hey  As you identified the problem is that merge_sort has no way of knowing the basis of sorting. You could change merge_sort to take in an additional parameter that returns the key for each element in the sequence just like sorted does:Then change the comparison to call passed function instead of comparing elements directly:And finally pass key to recursive calls:With these changes it will work as you expect:One thing to note though is that results are not identical with sorted since merge_sort is not stable:
Hey  Is there a reason you're avoiding creating some objects to manage this? If it were me, I'd go objects an

In [36]:
import spacy
from gensim.corpora import Dictionary
from gensim.models.tfidfmodel import TfidfModel
from gensim import corpora, models, similarities
from gensim.matutils import sparse2full
import numpy as np
import math

In [37]:
data['answer list'] = data['answer list'].str.split()
data['question list'] = data['question list'].str.split()

In [ ]:
tf = data.dropna()

In [38]:
data

,answer,link,code,ans length,Topic 0,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5,Topic 6,Topic 7,answer list,id,score,question list
4728,You can either use update:output:Or loc:output:,NaN,update\r\nEmpty_DF.update(ROI_DF)\r\n\r\n ...,6,0,0,0,0,0,0,0,1,"[either, use, updat, output, loc, output, upda...",58276538,2,"[merg, join, empti, datafram, anoth, fill, dat..."
4729,"This is perfect case for DataFrame.update, whi...",https://pandas.pydata.org/pandas-docs/stable/r...,DataFrame.update\r\nEmpty_DF.update(ROI_DF)\r\...,44,1,0,0,1,0,0,0,0,"[perfect, case, datafram, updat, align, indice...",58276538,3,"[merg, join, empti, datafram, anoth, fill, dat..."
4730,In your case reindex_like,NaN,reindex_like\r\nyourdf=ROI_DF.reindex_like(Emp...,4,0,0,0,0,0,0,0,0,"[case, reindex, likereindex, like, yourdf, roi...",58276538,1,"[merg, join, empti, datafram, anoth, fill, dat..."
60589,You can put the different column into index an...,NaN,"pd.concat([plantsFrame.set_index(['plants']), ...",12,0,0,0,0,0,0,0,1,"[put, differ, column, index, concat, output, p...",50727548,0,"[merg, multipl, datafram, number, column, one,..."
27294,Your desired outputs are no longer really mode...,https://stackoverflow.com/a/35019122/12197595,NaN,46,0,1,0,0,1,0,0,1,NaN,58678560,0,"[merg, similar, data, custom, field, use, seri..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39444,Use this code :Ouptut:,NaN,"list1= [['user1', 186, 'Feb 2017, Apr 2017', 5...",4,0,0,0,1,0,0,1,0,"[use, code, ouptut, list, user, feb, apr, user...",46362972,1,"[merg, list, list, python, list, list, desir, ..."
18975,This might help,NaN,"desiredlist = list(map(lambda y:[lst1,y],lst2)...",3,0,0,0,0,0,0,0,0,"[might, helpdesiredlist, list, map, lambda, ls...",48641350,0,"[merg, two, list, list, multipl, list, given, ..."
18976,You could achieve your desired output with ite...,NaN,itertools.zip_longest\r\nfillvalue\r\n>>> from...,31,0,0,0,0,0,0,0,1,"[could, achiev, desir, output, itertool, zip, ...",48641350,1,"[merg, two, list, list, multipl, list, given, ..."
18977,"Or you can use use append, but you need to cre...",NaN,lst3 = []\r\nfor elem in lst2:\r\n theNew =...,16,1,0,0,0,0,0,0,1,"[use, use, append, need, creat, new, copi, lst...",48641350,1,"[merg, two, list, list, multipl, list, given, ..."


In [39]:
answers = data['answer list']
questions = data['question list']

In [43]:
for answer in answers:

4728     [either, use, updat, output, loc, output, upda...
4729     [perfect, case, datafram, updat, align, indice...
4730     [case, reindex, likereindex, like, yourdf, roi...
60589    [put, differ, column, index, concat, output, p...
27294                                                  NaN
                               ...                        
39444    [use, code, ouptut, list, user, feb, apr, user...
18975    [might, helpdesiredlist, list, map, lambda, ls...
18976    [could, achiev, desir, output, itertool, zip, ...
18977    [use, use, append, need, creat, new, copi, lst...
18978                                                  NaN
Name: answer list, Length: 181, dtype: object

In [41]:
id2word = corpora.Dictionary(questions)
corpus = [id2word.doc2bow(question) for question in questions]

In [42]:
id2word = corpora.Dictionary(answers)
corpus = [id2word.doc2bow(answer) for answer in answers]

TypeError: 'float' object is not iterable

In [ ]:
#text2vec methods
class text2vec():
    def __init__(self, doc_list):
        #Initialize
        self.doc_list = doc_list
        self.nlp, self.docs, self.docs_dict = self._preprocess(self.doc_list)
    
    # Functions to lemmatise docs
    def _keep_token(self, t):
        return (t.is_alpha and 
                not (t.is_space or t.is_punct or 
                     t.is_stop or t.like_num))
    def _lemmatize_doc(self, doc):
        return [ t.lemma_ for t in doc if self._keep_token(t)]


    #Gensim to create a dictionary and filter out stop and infrequent words (lemmas).
    def _get_docs_dict(self, docs):
        docs_dict = Dictionary(docs)
        #CAREFUL: For small corpus please carefully modify the parameters for filter_extremes, or simply comment it out.
        docs_dict.filter_extremes(no_below=5, no_above=0.2)
        docs_dict.compactify()
        return docs_dict

    # Preprocess docs
    def _preprocess(self, doc_list):
        #Load spacy model
        nlp  = spacy.load('en')
        #lemmatise docs
        docs = [self._lemmatize_doc(nlp(doc)) for doc in doc_list] 
        #Get docs dictionary
        docs_dict = self._get_docs_dict(docs)
        return nlp, docs, docs_dict


    # Gensim can again be used to create a bag-of-words representation of each document,
    # build the TF-IDF model, 
    # and compute the TF-IDF vector for each document.
    def _get_tfidf(self, docs, docs_dict):
        docs_corpus = [docs_dict.doc2bow(doc) for doc in docs]
        model_tfidf = TfidfModel(docs_corpus, id2word=docs_dict)
        docs_tfidf  = model_tfidf[docs_corpus]
        docs_vecs   = np.vstack([sparse2full(c, len(docs_dict)) for c in docs_tfidf])
        return docs_vecs


    #Get avg w2v for one document
    def _document_vector(self, doc, docs_dict, nlp):
        # remove out-of-vocabulary words
        doc_vector = [nlp(word).vector for word in doc if word in docs_dict.token2id]
        return np.mean(doc_vector, axis=0)


    # Get a TF-IDF weighted Glove vector summary for document list
    # Input: a list of documents, Output: Matrix of vector for all the documents
    def tfidf_weighted_wv(self):
        #tf-idf
        docs_vecs   = self._get_tfidf(self.docs, self.docs_dict)

        #Load glove embedding vector for each TF-IDF term
        tfidf_emb_vecs = np.vstack([self.nlp(self.docs_dict[i]).vector for i in range(len(self.docs_dict))])

        #To get a TF-IDF weighted Glove vector summary of each document, 
        #we just need to matrix multiply docs_vecs with tfidf_emb_vecs
        docs_emb = np.dot(docs_vecs, tfidf_emb_vecs)

        return docs_emb

    # Get average vector for document list
    def avg_wv(self):
        docs_vecs = np.vstack([self._document_vector(doc, self.docs_dict, self.nlp) for doc in self.docs])
        return docs_vecs

    # Get TF-IDF vector for document list
    def get_tfidf(self):
        docs_corpus = [self.docs_dict.doc2bow(doc) for doc in self.docs]
        model_tfidf = TfidfModel(docs_corpus, id2word=self.docs_dict)
        docs_tfidf  = model_tfidf[docs_corpus]
        docs_vecs   = np.vstack([sparse2full(c, len(self.docs_dict)) for c in docs_tfidf])
        return docs_vecs


    # Get Latent Semantic Indexing(LSI) vector for document list
    def get_lsi(self, num_topics=300):
        docs_corpus = [self.docs_dict.doc2bow(doc) for doc in self.docs]
        model_lsi = models.LsiModel(docs_corpus, num_topics, id2word=self.docs_dict)
        docs_lsi  = model_lsi[docs_corpus]
        docs_vecs   = np.vstack([sparse2full(c, len(self.docs_dict)) for c in docs_lsi])
        return docs_vecs

    # Get Random Projections(RP) vector for document list
    def get_rp(self):
        docs_corpus = [self.docs_dict.doc2bow(doc) for doc in self.docs]
        model_rp = models.RpModel(docs_corpus, id2word=self.docs_dict)
        docs_rp  = model_rp[docs_corpus]
        docs_vecs   = np.vstack([sparse2full(c, len(self.docs_dict)) for c in docs_rp])
        return docs_vecs

    # Get Latent Dirichlet Allocation(LDA) vector for document list
    def get_lda(self, num_topics=100):
        docs_corpus = [self.docs_dict.doc2bow(doc) for doc in self.docs]
        model_lda = models.LdaModel(docs_corpus, num_topics, id2word=self.docs_dict)
        docs_lda  = model_lda[docs_corpus]
        docs_vecs   = np.vstack([sparse2full(c, len(self.docs_dict)) for c in docs_lda])
        return docs_vecs

    # Get Hierarchical Dirichlet Process(HDP) vector for document list
    def get_hdp(self):
        docs_corpus = [self.docs_dict.doc2bow(doc) for doc in self.docs]
        model_hdp = models.HdpModel(docs_corpus, id2word=self.docs_dict)
        docs_hdp  = model_hdp[docs_corpus]
        docs_vecs   = np.vstack([sparse2full(c, len(self.docs_dict)) for c in docs_hdp])
        return docs_vecs

    
    
#Similarity Calculation methods
class simical():
    def __init__(self, vec1, vec2):
        self.vec1 = vec1
        self.vec2 = vec2

    def _VectorSize(self, vec) :
        return math.sqrt(sum(math.pow(v,2) for v in vec))

    def _InnerProduct(self) :
        return sum(v1*v2 for v1,v2 in zip(self.vec1,self.vec2))

    def _Theta(self) :
        return math.acos(self.Cosine()) + 10
   
    def _Magnitude_Difference(self) :
        return abs(self._VectorSize(self.vec1) - self._VectorSize(self.vec2))
    
    def Euclidean(self) :
        return math.sqrt(sum(math.pow((v1-v2),2) for v1,v2 in zip(self.vec1, self.vec2)))
    
    def Cosine(self) :
        result = self._InnerProduct() / (self._VectorSize(self.vec1) * self._VectorSize(self.vec2))
        return result

    def Triangle(self) :
        theta = math.radians(self._Theta())
        return (self._VectorSize(self.vec1) * self._VectorSize(self.vec2) * math.sin(theta)) / 2

    def Sector(self) :
        ED = self.Euclidean()
        MD = self._Magnitude_Difference()
        theta = self._Theta()
        return math.pi * math.pow((ED+MD),2) * theta/360

    def TS_SS(self) :
        return self.Triangle() * self.Sector()